# <center>Implementing LDA in Python</center>

<center>Dr. W.J.B. Mattingly</center>

<center>Smithsonian Data Science Lab and United States Holocaust Memorial Museum</center>

<center>February 2021</center>

## Key Concepts in this Notebook

## Introduction

TODO list :

- Check this : Latent-Dirichlet-Allocation-LDA
- INSTA automatic comment extraction
- install on ania laptop

## Importing the Required Libraries

In [1]:
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#1introduction
import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Export panda
import pandas as pd

## Preparing the Data

In [2]:
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f) 
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)


In [3]:
from nltk.corpus import stopwords
stopwords = stopwords.words("english")

In [4]:
print (stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [4]:
# Load data from INSTAGRAM

filename = "all_data.xlsx"
xl = pd.ExcelFile(filename)
table = xl.parse()

In [50]:
#####################
# pre-treat the data
#####################

# emoji_list = []
# counter = 0
# for elt in table.get("COMMENT")[:]:
#     for c in elt :
#         if len(c.encode('utf8')) > 1:
#             if not(c in emoji_list) :
#                 emoji_list.append(c)
#                 counter += 1

#                 counter = 0


# Extract comments as a list of separate words                
# word_list, world_list_clean, emoji_list = [], [], [] 

# for elt in table.get("COMMENT")[:]:
#     word_list.append(elt.split())
    
# for line in word_list:
#     new_line = []
#     for word in line :
#         new_word = ""
#         for id,character in enumerate(word):
#             if character.encode("utf-8")[0] > 122 :
#                 if not (character in emoji_list):
#                     emoji_list.append(character)
#                 # TODO : add a new word corresponding to the emoji
#                 # new_word += "FIRERERE"
#             else:
#                 new_word += character
#         if len(new_word) > 0:
#             new_line.append(new_word)
#     if len(new_line) > 0 :
#          world_list_clean.append(new_line)              

In [5]:
corpus,emoji_list = [], []
for line in table.get("COMMENT")[:]:
    new_line = ""
    for id,character in enumerate(line):
        if character.encode("utf-8")[0] > 122 :
            if not (character in emoji_list):
                emoji_list.append(character)
            # TODO : add a new word corresponding to the emoji
            # new_word += "FIRERERE"
        else:
            new_line += character
    if len(new_line) > 0:
        corpus.append(new_line)

## Pre-process using 

https://github.com/sethns/Latent-Dirichlet-Allocation-LDA-/blob/main/Topic%20Modeling%20_%20Extracting%20Topics_%20Using%20Genism.ipynb?fbclid=IwAR2renok7J_JmL8yWYHWV3cu69zdNiWpmwv-qz2PxWLaBNIHahFQuRyAg_U


2. Text Preprocessing

Steps to preprocess text data:

   -  Convert the text into lowercase
   - Split text into words
   -  Remove the stop loss words
   -  Remove the Punctuation, any symbols and special characters
   - Normalize the word (I'll be using Lemmatization for normalization)



In [11]:
##################
# PRE-PROCESS 2.0
# using LDA
##################
# stop loss words 
# stop = set(stopwords.words('english'))
import string
from nltk.stem.wordnet import WordNetLemmatizer

# punctuation 
exclude = set(string.punctuation) 

# lemmatization
lemma = WordNetLemmatizer() 

# One function for all the steps:
def clean(doc):
    
    # convert text into lower case + split into words
    stop_free = " ".join([i for i in doc.lower().split() if i not in stopwords])
    
    # remove any stop words present
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)  
    
    # remove punctuations + normalize the text
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())  
    return normalized

# clean data stored in a new list
clean_corpus = [clean(doc).split() for doc in corpus] 

In [115]:
##################
# PRE-PROCESS 2.1
##################

# def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
#     nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
#     texts_out = []
#     for text in texts:
#         doc = nlp(text)
#         new_text = []
#         for token in doc:
#             if token.pos_ in allowed_postags:
#                 new_text.append(token.lemma_)
#         final = " ".join(new_text)
#         texts_out.append(final)
#     return (texts_out)

# def gen_words(texts):
#     final = []
#     for text in texts:
#         new = gensim.utils.simple_preprocess(text, deacc=True)
#         final.append(new)
#     return (final)

# lemmatized_texts = lemmatization(corpus)
# clean_corpus = gen_words(lemmatized_texts)

# print (clean_corpus[0][0:20])

['beautiful']


## Data to matrix representation

In [13]:
# Version 1.

# id2word = corpora.Dictionary(clean_corpus)

# corpus2 = []
# for text in clean_corpus:
#     new = id2word.doc2bow(text)
#     corpus2.append(new)

# print (corpus2[0][0:20])

# word = id2word[[0][:1][0]]
# print (word)

[(0, 1)]
beautiful


In [15]:
# Version 2, more efficient

#BIGRAMS AND TRIGRAMS
data_words = clean_corpus
bigram_phrases = gensim.models.Phrases(clean_corpus, min_count=5, threshold=100)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=100)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return ([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

print (data_bigrams_trigrams[0][0:20])

#TF-IDF REMOVAL
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus2 = [id2word.doc2bow(text) for text in texts]
# print (corpus[0][0:20])

tfidf = TfidfModel(corpus2, id2word=id2word)

low_value = 0.03
words  = []
words_missing_in_tfidf = []
for i in range(0, len(corpus2)):
    bow = corpus2[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus2[i] = new_bow


['beautiful']


## Train the model

In [16]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus2,
                                           id2word=id2word,
                                           num_topics=8,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")


# Visualization of the data 

In [17]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus2, id2word, mds="mmds", R=30)
vis

/home/thomas_cbrs/.local/lib/python3.8/site-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.379794  0.061971       1        1  22.471196
5      0.202921  0.273433       2        1  16.946675
0      0.357545 -0.057384       3        1  13.130845
4      0.126201 -0.026070       4        1  10.685817
6     -0.191436 -0.246083       5        1  10.169309
1     -0.074058  0.292598       6        1   9.342910
2      0.080435 -0.316096       7        1   9.324148
3     -0.121814  0.017632       8        1   7.929100, topic_info=           Term        Freq       Total Category  logprob  loglift
37           hm  112.000000  112.000000  Default  30.0000  30.0000
22       please  103.000000  103.000000  Default  29.0000  29.0000
1          love   71.000000   71.000000  Default  28.0000  28.0000
1221     animal   44.000000   44.000000  Default  27.0000  27.0000
127           u   81.000000   81.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
475        life    2.526986    3.134116   Topic8  -5.3245   2.3193
859     comment    2.414537    3.021986   Topic8  -5.3700   2.3102
1243   addition    2.372119    2.992337   Topic8  -5.3877   2.3024
283   available    6.705092   10.670966   Topic8  -4.3487   2.0700
37           hm    4.733408  112.153720   Topic8  -4.6969  -0.6306

[311 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
826       2  0.324713         2
826       3  0.649426         2
104       5  0.967294      able
1373      4  0.919226     abuse
1243      8  0.668374  addition
...     ...       ...       ...
369       5  0.943021       wow
1557      3  0.970077     write
67        4  0.787157       yes
23        7  0.950762       you
670       5  0.943873     yours

[305 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 6, 1, 5, 7, 2, 3, 4])